### 전처리 2단계  
1. 점포 데이터에 폐업률 추가  
2. 누락된 상권이 있는지    
3. 결측치 , 이상치 확인  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc

In [2]:
# 생활인구
living_pop = pd.read_csv("preprocessed_data/living_pop_1.csv",encoding = "UTF-8")

# 집객시설
jipgaek = pd.read_csv("preprocessed_data/jipgaek_1.csv",encoding="UTF-8")

# 점포
jumpo = pd.read_csv("preprocessed_data/jumpo_1.csv",encoding="UTF-8")

# 아파트
apt = pd.read_csv("preprocessed_data/apt_1.csv",encoding="UTF-8") 

# 추정매출 
est_sale =  pd.read_csv("preprocessed_data/est_sale_1.csv",encoding="UTF-8") 

# 상권변화지표
change_factor = pd.read_csv("preprocessed_data/change_factor_1.csv",encoding="UTF-8")

#### 생활인구 전처리  

In [10]:
# 생활인구부터 상권변화지표 순서로 전처리 진행
# 같은년도의 같은 상권코드명을 가지고 있어도, 분기별로 다른 총생활인구수가 다르기 때문에
# 연도별로 계산하는 해당 프로젝트에서는 분기별 데이터를 년도별 데이터로 묶을 예정이다.
living_pop = living_pop.groupby(['기준_년_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명'],as_index=False)[['총_생활인구_수']].mean()
living_pop

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수
0,2017,A,골목상권,1000001,계동길,337476.375
1,2017,A,골목상권,1000002,난계로27길,1026880.000
2,2017,A,골목상권,1000003,돈화문로11가길,801396.000
3,2017,A,골목상권,1000004,명륜길,759851.875
4,2017,A,골목상권,1000005,백석동길,23389.000
...,...,...,...,...,...,...
11498,2022,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,4268885.750
11499,2022,U,관광특구,1001493,동대문패션타운 관광특구,3129478.000
11500,2022,U,관광특구,1001494,종로?청계 관광특구,3864423.000
11501,2022,U,관광특구,1001495,잠실 관광특구,3469611.000


In [50]:
# 특수문자 제거
import re
def cleanText(Data): # 괄호와 _을 제외한 모든 특수문자
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\\\[\]\<\>`\'…》]','', Data)
    return text 
#print(cleanText("$%!^!#@*&^$(@#^$&___  hello_world  ___)!@%$#$!^@%$")) #-> (___  hello_world  ___)
temp=[]
for i in living_pop['상권_코드_명']:
    temp.append(cleanText(i))
living_pop['상권_코드_명']=temp

In [58]:
living_pop

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수
0,2017,A,골목상권,1000001,계동길,337476.375
1,2017,A,골목상권,1000002,난계로27길,1026880.000
2,2017,A,골목상권,1000003,돈화문로11가길,801396.000
3,2017,A,골목상권,1000004,명륜길,759851.875
4,2017,A,골목상권,1000005,백석동길,23389.000
...,...,...,...,...,...,...
11498,2022,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,4268885.750
11499,2022,U,관광특구,1001493,동대문패션타운 관광특구,3129478.000
11500,2022,U,관광특구,1001494,종로청계 관광특구,3864423.000
11501,2022,U,관광특구,1001495,잠실 관광특구,3469611.000


In [60]:
# 기능상실 적힌 상권코드명 row 제거
drop_list = []
for i in living_pop['상권_코드_명']:
    if('기능상실' in i and i not in drop_list):
        drop_list.append(i)
    else: continue

for i in drop_list:
    living_pop.drop(living_pop[(living_pop['상권_코드_명']==i)].index,inplace = True)

In [ ]:
# 값이 없는 부분을 key error 나타나기 전에 제거

temp=[]
for i in range(len(living_pop)):
    try: 
        print(living_pop['상권_코드_명'][i] ,living_pop['상권_코드'][i]) 
    except KeyError:
        temp.append(i)
# 셀 출력이 길어서 지웠습니다

In [101]:
for i in temp:
    living_pop.drop(index=living_pop.iloc[i].name)

In [110]:
living_pop=living_pop.reset_index(drop=True)

In [131]:
# 상권_코드와 상권_코드명의 개수가 일치하지 않은 dataframe이 여럿 있지만 어떻게 찾아야하는지 미지수..
print(living_pop['상권_코드_명'].nunique())
print(living_pop['상권_코드'].nunique()) 
print(jipgaek['상권_코드_명'].nunique())
print(jipgaek['상권_코드'].nunique())
print(jumpo['상권_코드_명'].nunique())
print(jumpo['상권_코드'].nunique())
print(apt['상권_코드_명'].nunique())
print(apt['상권_코드'].nunique())
print(est_sale['상권_코드_명'].nunique())
print(est_sale['상권_코드'].nunique())
print(change_factor['상권_코드_명'].nunique())
print(change_factor['상권_코드'].nunique())

2981
3153
1647
1647
1668
1667
1528
1528
1658
1657
1668
1668


#### 집객시설 전처리  

In [149]:
#집객시설 전처리
jipgaek= jipgaek.groupby(['기준_년_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명'],as_index=False)[['집객시설_수']].mean()
#특수문자 제거
import re
def cleanText(Data): # 괄호와 _을 제외한 모든 특수문자
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\\\[\]\<\>`\'…》]','', Data)
    return text 
temp=[]
for i in jipgaek['상권_코드_명']:
    temp.append(cleanText(i))
jipgaek['상권_코드_명']=temp

# 기능상실 적힌 상권코드명 row 제거
drop_list = []
for i in jipgaek['상권_코드_명']:
    if('기능상실' in i and i not in drop_list):
        drop_list.append(i)
    else: continue

for i in drop_list:
    jipgaek.drop(jipgaek[(jipgaek['상권_코드_명']==i)].index,inplace = True)

# 값이 없는 부분을 key error 나타나기 전에 제거
temp=[]
for i in range(len(jipgaek)):
    try: 
        print(jipgaek['상권_코드_명'][i] ,jipgaek['상권_코드'][i]) 
    except KeyError:
        temp.append(i)
for i in temp:
    jipgaek.drop(index=jipgaek.iloc[i].name)
# index reset
jipgaek=jipgaek.reset_index(drop=True)

In [165]:
jipgaek

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수
0,2017,A,골목상권,2110001,이북5도청사,22.5
1,2017,A,골목상권,2110002,독립문역 1번,31.5
2,2017,A,골목상권,2110003,세검정초등학교,48.0
3,2017,A,골목상권,2110004,대신고등학교,44.5
4,2017,A,골목상권,2110005,세검정,44.5
...,...,...,...,...,...,...
9748,2022,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,426.0
9749,2022,U,관광특구,1001493,동대문패션타운 관광특구,149.0
9750,2022,U,관광특구,1001494,종로청계 관광특구,243.0
9751,2022,U,관광특구,1001495,잠실 관광특구,149.0


#### 아파트 전처리  

In [ ]:
# 아파트 전처리
apt= apt.groupby(['기준_년_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명'],as_index=False)[['아파트_평균_면적','아파트_평균_시가']].mean()

#특수문자 제거
import re
def cleanText(Data): # 괄호와 _을 제외한 모든 특수문자
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\\\[\]\<\>`\'…》]','', Data)
    return text 
temp=[]
for i in apt['상권_코드_명']:
    temp.append(cleanText(i))
apt['상권_코드_명']=temp

# 기능상실 적힌 상권코드명 row 제거
drop_list = []
for i in apt['상권_코드_명']:
    if('기능상실' in i and i not in drop_list):
        drop_list.append(i)
    else: continue

for i in drop_list:
    apt.drop(apt[(apt['상권_코드_명']==i)].index,inplace = True)

# 값이 없는 부분을 key error 나타나기 전에 제거
temp=[]
for i in range(len(apt)):
    try: 
        print(apt['상권_코드_명'][i] ,apt['상권_코드'][i]) 
    except KeyError:
        temp.append(i)
for i in temp:
    apt.drop(index=apt.iloc[i].name)
# index reset
apt = apt.reset_index(drop=True)

In [201]:
apt

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,아파트_평균_면적,아파트_평균_시가
0,2017,A,골목상권,2110001,이북5도청사,124.0,376996219.5
1,2017,A,골목상권,2110002,독립문역 1번,78.0,311503096.0
2,2017,A,골목상권,2110003,세검정초등학교,89.0,223873171.0
3,2017,A,골목상권,2110004,대신고등학교,50.5,122651126.0
4,2017,A,골목상권,2110005,세검정,88.5,225876911.5
...,...,...,...,...,...,...,...
9071,2022,R,전통시장,2130326,고덕 골목형상점가,53.0,633301800.0
9072,2022,U,관광특구,1001491,이태원 관광특구,79.0,439233880.0
9073,2022,U,관광특구,1001493,동대문패션타운 관광특구,45.0,259406778.0
9074,2022,U,관광특구,1001494,종로청계 관광특구,37.0,141690132.0


#### 상권변화지표 전처리  

In [ ]:
#상권변화지표 전처리
change_factor = change_factor.groupby(['기준_년_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명','상권_변화_지표','상권_변화_지표_명'],as_index=False)[['운영_영업_개월_평균','폐업_영업_개월_평균','서울_운영_영업_개월_평균','서울_폐업_영업_개월_평균']].mean()

#특수문자 제거
import re
def cleanText(Data): # 괄호와 _을 제외한 모든 특수문자
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\\\[\]\<\>`\'…》]','', Data)
    return text 
temp=[]
for i in change_factor['상권_코드_명']:
    temp.append(cleanText(i))
change_factor['상권_코드_명']=temp

# 기능상실 적힌 상권코드명 row 제거
drop_list = []
for i in change_factor['상권_코드_명']:
    if('기능상실' in i and i not in drop_list):
        drop_list.append(i)
    else: continue

for i in drop_list:
    change_factor.drop(change_factor[(change_factor['상권_코드_명']==i)].index,inplace = True)

# 값이 없는 부분을 key error 나타나기 전에 제거
temp=[]
for i in range(len(change_factor)):
    try: 
        print(change_factor['상권_코드_명'][i] ,change_factor['상권_코드'][i]) 
    except KeyError:
        temp.append(i)
for i in temp:
    change_factor.drop(index=change_factor.iloc[i].name)
# index reset
change_factor = change_factor.reset_index(drop=True)

In [211]:
change_factor

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2017,A,골목상권,2110001,이북5도청사,LH,상권확장,83.75,60.25,98.75,51.25
1,2017,A,골목상권,2110002,독립문역 1번,HH,정체,117.75,55.00,98.75,51.25
2,2017,A,골목상권,2110003,세검정초등학교,HH,정체,107.75,55.00,98.75,51.25
3,2017,A,골목상권,2110004,대신고등학교,HH,정체,153.25,68.25,98.75,51.25
4,2017,A,골목상권,2110005,세검정,HH,정체,105.75,56.50,98.75,51.25
...,...,...,...,...,...,...,...,...,...,...,...
11330,2022,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,HH,정체,167.75,65.50,121.25,55.50
11331,2022,U,관광특구,1001493,동대문패션타운 관광특구,HH,정체,147.00,64.00,121.25,55.50
11332,2022,U,관광특구,1001494,종로청계 관광특구,HH,정체,162.50,74.75,121.25,55.50
11333,2022,U,관광특구,1001495,잠실 관광특구,LL,다이나믹,117.00,54.25,121.25,55.50


## 점포, 추정매출 전처리는 서비스 업종 코드를 나눠야 하기 때문에 다른 전처리 과정이 필요함  
#### 점포 전처리  

In [226]:
jumpo=jumpo.assign(구분=0)
jumpo

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,구분
0,2017,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,12,0
1,2017,A,골목상권,2110885,영락고등학교,CS200016,당구장,4,0
2,2017,A,골목상권,2110885,영락고등학교,CS200015,세무사사무소,2,0
3,2017,A,골목상권,2110885,영락고등학교,CS200012,법무사사무소,1,0
4,2017,A,골목상권,2110885,영락고등학교,CS200009,동물병원,3,0
...,...,...,...,...,...,...,...,...,...
3862295,2022,A,골목상권,2110732,가산초등학교(세일중학교),CS200034,여관,16,0
3862296,2022,A,골목상권,2110732,가산초등학교(세일중학교),CS200036,고시원,6,0
3862297,2022,A,골목상권,2110732,가산초등학교(세일중학교),CS200037,노래방,26,0
3862298,2022,A,골목상권,2110732,가산초등학교(세일중학교),CS200028,미용실,54,0


In [227]:
음식업_및_주점업 = []
식료품_도소매업=[]
보건업=[]
전문_서비스업 = []
교육_및_사업자원_서비스업=[]
스포츠_및_오락_서비스업=[]
개인_서비스업=[]
기타_서비스업=[]
생활용품_도소매업=[]
가전제품_및_통신기기_도소매업=[]
중고상품_도소매업=[]
기타_도소매업=[]
수리업=[]
숙박업=[]
부동산_및_임대업=[]
제조업=[]

In [228]:
for i ,j in enumerate(jumpo['서비스_업종_코드_명']):
    if( j in ['한식음식점','호프-간이주점','치킨전문점','패스트푸드점','제과점','양식음식점','일식음식점','중국음식점','분식전문점','커피-음료']):
        jumpo['구분'][i]='음식점_및_주점업'
    elif(j in ['반찬가게','청과상','수산물판매','육류판매','미곡판매','주류판매','편의점','슈퍼마켓']):
        jumpo['구분'][i]='식료품_도소매업'
    elif(j in ['동물병원','한의원','치과의원','일반의원']):
        jumpo['구분'][i]='보건업' # 얘네들은 자격이 필요한 애들이라 나중에 싹 버릴거임
    elif(j in ['세무사사무소','법무사사무소','인테리어','기타법무서비스','회계사사무소','통번역서비스','변리사사무소']):
        jumpo['구분'][i]='전문_서비스업'#얘네들도?
    elif(j in ['예술학원','외국어학원','일반교습학원','컴퓨터학원','여행사']):
        jumpo['구분'][i]='교육_및_사업자원_서비스업'
    elif(j in ['골프연습장','스포츠클럽', '볼링장', '스포츠 강습', 'PC방','기타오락장','노래방','복권방','DVD방','전자게임장']):
        jumpo['구분'][i]='스포츠_및_오락_서비스업'
    elif(j in ['세탁소','피부관리실','미용실','사진관','녹음실','네일숍']):
        jumpo['구분'][i]='개인_서비스업'
    elif(j in ['건축물청소','독서실']):
        jumpo['구분'][i]='기타_서비스업'
    elif(j in ['일반의류','운동/경기용품','유아의류','한복점','서적']):
        jumpo['구분'][i]='생활용품_도소매업'
    elif(j in ['핸드폰','컴퓨터및주변장치판매','가전제품']):
        jumpo['구분'][i]='가전제품_및_통신기기_도소매업'
    elif(j in ['재생용품 판매점', '중고차판매','중고가구']):
        jumpo['구분'][i]='중고상품_도소매업'
    elif(j in ['화초', '섬유제품','완구','미용재료','화장품','문구','주유소','전자상거래업','애완동물']):
        jumpo['구분'][i]='기타_도소매업'
    elif(j in ['모터사이클및부품','자동차부품','가전제품수리','모터사이클수리','자동차미용','자동차수리','통신기기수리','가방','신발']):
        jumpo['구분'][i]='수리업'
    elif(j in ['고시원','여관','게스트하우스']):
        jumpo['구분'][i]='숙박업'
    elif(j in ['부동산중개업','가정용품임대','비디오/서적임대','의류임대']):
        jumpo['구분'][i]='부동산_및_임대업'
    else:
        jumpo['구분'][i]='제조업'

/var/folders/b7/dwjv2sgs2_s0329_m054lq800000gn/T/ipykernel_1071/3964629994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jumpo['구분'][i]='음식점_및_주점업'


In [236]:
for i in range(jumpo['구분'].nunique()):
    print(jumpo['구분'].unique()[i],len(jumpo[jumpo['구분']==jumpo['구분'].unique()[i]]))

음식점_및_주점업 497488
제조업 534504
전문_서비스업 170590
보건업 172223
스포츠_및_오락_서비스업 304429
교육_및_사업자원_서비스업 202386
기타_도소매업 392781
중고상품_도소매업 44582
수리업 315285
생활용품_도소매업 201135
식료품_도소매업 360232
가전제품_및_통신기기_도소매업 146560
기타_서비스업 71200
숙박업 69759
부동산_및_임대업 113188
개인_서비스업 265958


In [237]:
jumpo

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,구분
0,2017,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,12,음식점_및_주점업
1,2017,A,골목상권,2110885,영락고등학교,CS200016,당구장,4,제조업
2,2017,A,골목상권,2110885,영락고등학교,CS200015,세무사사무소,2,전문_서비스업
3,2017,A,골목상권,2110885,영락고등학교,CS200012,법무사사무소,1,전문_서비스업
4,2017,A,골목상권,2110885,영락고등학교,CS200009,동물병원,3,보건업
...,...,...,...,...,...,...,...,...,...
3862295,2022,A,골목상권,2110732,가산초등학교(세일중학교),CS200034,여관,16,숙박업
3862296,2022,A,골목상권,2110732,가산초등학교(세일중학교),CS200036,고시원,6,숙박업
3862297,2022,A,골목상권,2110732,가산초등학교(세일중학교),CS200037,노래방,26,스포츠_및_오락_서비스업
3862298,2022,A,골목상권,2110732,가산초등학교(세일중학교),CS200028,미용실,54,개인_서비스업


In [244]:
#jumpo = jumpo.groupby(['기준_년_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명','상권_변화_지표','상권_변화_지표_명'],as_index=False)[['운영_영업_개월_평균','폐업_영업_개월_평균','서울_운영_영업_개월_평균','서울_폐업_영업_개월_평균']].mean()
jumpo=jumpo.drop(['서비스_업종_코드','서비스_업종_코드_명'],axis=1)

In [239]:
jumpo_temp = jumpo.groupby(['기준_년_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명','구분'],as_index=False)[['점포_수']].mean()


In [273]:
jumpo_temp[jumpo_temp['구분']=='음식점_및_주점업'].sort_values(by='점포_수',ascending=False).head(10)

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,구분,점포_수
8848,2017,A,골목상권,2110566,경의선책거리,음식점_및_주점업,9049
1325,2017,A,골목상권,2110086,이태원역 북측,음식점_및_주점업,7212
8047,2017,A,골목상권,2110514,서대문우체국,음식점_및_주점업,6645
11110,2017,A,골목상권,2110710,남구로역 3번,음식점_및_주점업,6639
15694,2017,A,골목상권,2111004,삼전역 1번,음식점_및_주점업,6409
13620,2017,A,골목상권,2110869,"신림역 5번(신림동주민센터, 신림동별빛거리)",음식점_및_주점업,6379
8742,2017,A,골목상권,2110559,성산중학교,음식점_및_주점업,6141
11222,2017,A,골목상권,2110717,남구로역 2번,음식점_및_주점업,6085
8817,2017,A,골목상권,2110564,홍대입구역 3번,음식점_및_주점업,6029
16556,2017,A,골목상권,2111059,강동역 4번(강풀만화거리),음식점_및_주점업,5897


In [263]:
#jumpo_temp[jumpo_temp['구분']=='개인_서비스업']
print(jumpo_temp[(jumpo_temp['상권_코드_명']=='이북5도청사') & (jumpo_temp['구분']=='제조업')])  # and가 아니고 &를 써야함
print(jumpo[(jumpo['상권_코드_명']=='이북5도청사') & (jumpo['구분']=='제조업')& (jumpo['기준_년_코드']==2017)]['점포_수'].sum())
print(jumpo[(jumpo['상권_코드_명']=='이북5도청사') & (jumpo['구분']=='제조업')& (jumpo['기준_년_코드']==2018)]['점포_수'].sum())
print(jumpo[(jumpo['상권_코드_명']=='이북5도청사') & (jumpo['구분']=='제조업')& (jumpo['기준_년_코드']==2019)]['점포_수'].sum())
print(jumpo[(jumpo['상권_코드_명']=='이북5도청사') & (jumpo['구분']=='제조업')& (jumpo['기준_년_코드']==2020)]['점포_수'].sum())
print(jumpo[(jumpo['상권_코드_명']=='이북5도청사') & (jumpo['구분']=='제조업')& (jumpo['기준_년_코드']==2021)]['점포_수'].sum())
print(jumpo[(jumpo['상권_코드_명']=='이북5도청사') & (jumpo['구분']=='제조업')& (jumpo['기준_년_코드']==2022)]['점포_수'].sum())
# 해당 결과를 보면 , 제대로 구한 것을 알 수 있다.

        기준_년_코드 상권_구분_코드 상권_구분_코드_명    상권_코드 상권_코드_명   구분  점포_수
13         2017        A       골목상권  2110001  이북5도청사  제조업   128
24612      2018        A       골목상권  2110001  이북5도청사  제조업    59
49175      2019        A       골목상권  2110001  이북5도청사  제조업     4
71056      2020        A       골목상권  2110001  이북5도청사  제조업    57
95623      2021        A       골목상권  2110001  이북5도청사  제조업    59
120139     2022        A       골목상권  2110001  이북5도청사  제조업    60
128
59
4
57
59
60


In [281]:
#특수문자 제거
import re
def cleanText(Data): # 괄호와 _을 제외한 모든 특수문자
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\\\[\]\<\>`\'…》]','', Data)
    return text 
temp=[]
for i in jumpo_temp['상권_코드_명']:
    temp.append(cleanText(i))
jumpo_temp['상권_코드_명']=temp
jumpo = jumpo_temp

In [283]:
# 기능상실 적힌 상권코드명 row 제거
drop_list = []
for i in jumpo['상권_코드_명']:
    if('기능상실' in i and i not in drop_list):
        drop_list.append(i)
    else: continue

for i in drop_list:
    jumpo.drop(jumpo[(jumpo['상권_코드_명']==i)].index,inplace = True)

In [ ]:
# 값이 없는 부분을 key error 나타나기 전에 제거
temp=[]
for i in range(len(jumpo)):
    try: 
        print(jumpo['상권_코드_명'][i] ,jumpo['상권_코드'][i]) 
    except KeyError:
        temp.append(i)
for i in temp:
    jumpo.drop(index=jumpo.iloc[i].name)
# index reset
jumpo=jumpo.reset_index(drop=True)

In [291]:
# 빼야 하는 것들 : 전문_서비스업 , 보건업 이렇게 두개
jumpo.drop(jumpo[jumpo['구분']=='전문_서비스업'].index,inplace=True)
jumpo.drop(jumpo[jumpo['구분']=='보건업'].index,inplace=True)

In [296]:
jumpo.reset_index(drop=True,inplace=True)

In [297]:
jumpo

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,구분,점포_수
0,2017,A,골목상권,2110001,이북5도청사,개인_서비스업,116
1,2017,A,골목상권,2110001,이북5도청사,교육_및_사업자원_서비스업,128
2,2017,A,골목상권,2110001,이북5도청사,기타_도소매업,98
3,2017,A,골목상권,2110001,이북5도청사,기타_서비스업,4
4,2017,A,골목상권,2110001,이북5도청사,부동산_및_임대업,156
...,...,...,...,...,...,...,...
126208,2022,U,관광특구,1001496,강남 마이스 관광특구,수리업,319
126209,2022,U,관광특구,1001496,강남 마이스 관광특구,스포츠_및_오락_서비스업,9
126210,2022,U,관광특구,1001496,강남 마이스 관광특구,식료품_도소매업,140
126211,2022,U,관광특구,1001496,강남 마이스 관광특구,음식점_및_주점업,549


#### 추정매출 전처리  

In [298]:
est_sale

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수
0,2017,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,816890247,28082
1,2017,A,골목상권,2110706,그라운드 고척,CS300031,가구,102416775,116
2,2017,A,골목상권,2110706,그라운드 고척,CS300029,애완동물,12257816,667
3,2017,A,골목상권,2110706,그라운드 고척,CS300028,화초,39851942,1615
4,2017,A,골목상권,2110706,그라운드 고척,CS300027,섬유제품,66164481,1067
...,...,...,...,...,...,...,...,...,...
2178583,2022,D,발달상권,2120240,개롱역,CS300010,반찬가게,64884644,4268
2178584,2022,D,발달상권,2120240,개롱역,CS300011,일반의류,11273198,150
2178585,2022,D,발달상권,2120240,개롱역,CS300016,안경,64011302,572
2178586,2022,D,발달상권,2120240,개롱역,CS300001,슈퍼마켓,4169784327,195988


In [303]:
est_sale['서비스_업종_코드_명'].unique()

array(['한식음식점', '가구', '애완동물', '화초', '섬유제품', '완구', '자전거 및 기타운송장비',
       '운동/경기용품', '화장품', '문구', '서적', '의약품', '시계및귀금속', '안경', '신발', '일반의류',
       '반찬가게', '청과상', '수산물판매', '육류판매', '핸드폰', '컴퓨터및주변장치판매', '가전제품', '편의점',
       '철물점', '조명용품', '스포츠클럽', '골프연습장', '당구장', '세무사사무소', '한의원', '치과의원',
       '일반의원', '스포츠 강습', '예술학원', '외국어학원', '일반교습학원', '커피-음료', '호프-간이주점',
       '분식전문점', '치킨전문점', '패스트푸드점', '제과점', '일식음식점', '중식음식점', '자동차부품',
       '인테리어', '슈퍼마켓', '사진관', '독서실', '양식음식점', '노래방', '고시원', '가전제품수리',
       '세탁소', '피부관리실', '미용실', '자동차미용', '자동차수리', 'PC방', '네일숍', '미곡판매',
       '여행사', '여관', '주유소', '부동산중개업', '전자상거래업', '회계사사무소', '악기', '의료기기',
       '가방', '법무사사무소', '한복점', '유아의류', '통신기기수리', '볼링장', '예술품', '중고차판매',
       '비디오/서적임대', '건축물청소', '변호사사무소'], dtype=object)

In [307]:
# 점포에서 없는 서비스업종인 '예술품'->'개인_서비스업', '변호사사무소'->'전문_서비스업', '당구장'->'스포츠_및_오락_서비스업'으로 분류하고 진행했습니다.
for i ,j in enumerate(est_sale['서비스_업종_코드_명']):
    if( j in ['한식음식점','호프-간이주점','치킨전문점','패스트푸드점','제과점','양식음식점','일식음식점','중국음식점','분식전문점','커피-음료']):
        est_sale['구분'][i]='음식점_및_주점업'
    elif(j in ['반찬가게','청과상','수산물판매','육류판매','미곡판매','주류판매','편의점','슈퍼마켓']):
        est_sale['구분'][i]='식료품_도소매업'
    elif(j in ['동물병원','한의원','치과의원','일반의원']):
        est_sale['구분'][i]='보건업' 
    elif(j in ['세무사사무소','법무사사무소','인테리어','기타법무서비스','회계사사무소','통번역서비스','변리사사무소','변호사사무소']):
        est_sale['구분'][i]='전문_서비스업'
    elif(j in ['예술학원','외국어학원','일반교습학원','컴퓨터학원','여행사']):
        est_sale['구분'][i]='교육_및_사업자원_서비스업'
    elif(j in ['골프연습장','스포츠클럽', '볼링장', '스포츠 강습', 'PC방','기타오락장','노래방','복권방','DVD방','전자게임장','당구장']):
        est_sale['구분'][i]='스포츠_및_오락_서비스업'
    elif(j in ['세탁소','피부관리실','미용실','사진관','녹음실','네일숍','예술품']):
        est_sale['구분'][i]='개인_서비스업'
    elif(j in ['건축물청소','독서실']):
        est_sale['구분'][i]='기타_서비스업'
    elif(j in ['일반의류','운동/경기용품','유아의류','한복점','서적']):
        est_sale['구분'][i]='생활용품_도소매업'
    elif(j in ['핸드폰','컴퓨터및주변장치판매','가전제품']):
        est_sale['구분'][i]='가전제품_및_통신기기_도소매업'
    elif(j in ['재생용품 판매점', '중고차판매','중고가구']):
        est_sale['구분'][i]='중고상품_도소매업'
    elif(j in ['화초', '섬유제품','완구','미용재료','화장품','문구','주유소','전자상거래업','애완동물']):
        est_sale['구분'][i]='기타_도소매업'
    elif(j in ['모터사이클및부품','자동차부품','가전제품수리','모터사이클수리','자동차미용','자동차수리','통신기기수리','가방','신발']):
        est_sale['구분'][i]='수리업'
    elif(j in ['고시원','여관','게스트하우스']):
        est_sale['구분'][i]='숙박업'
    elif(j in ['부동산중개업','가정용품임대','비디오/서적임대','의류임대']):
        est_sale['구분'][i]='부동산_및_임대업'
    else:
        est_sale['구분'][i]='제조업'

/var/folders/b7/dwjv2sgs2_s0329_m054lq800000gn/T/ipykernel_1071/1189712989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est_sale['구분'][i]='음식점_및_주점업'


In [309]:
est_sale

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,구분
0,2017,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,816890247,28082,음식점_및_주점업
1,2017,A,골목상권,2110706,그라운드 고척,CS300031,가구,102416775,116,제조업
2,2017,A,골목상권,2110706,그라운드 고척,CS300029,애완동물,12257816,667,기타_도소매업
3,2017,A,골목상권,2110706,그라운드 고척,CS300028,화초,39851942,1615,기타_도소매업
4,2017,A,골목상권,2110706,그라운드 고척,CS300027,섬유제품,66164481,1067,기타_도소매업
...,...,...,...,...,...,...,...,...,...,...
2178583,2022,D,발달상권,2120240,개롱역,CS300010,반찬가게,64884644,4268,식료품_도소매업
2178584,2022,D,발달상권,2120240,개롱역,CS300011,일반의류,11273198,150,생활용품_도소매업
2178585,2022,D,발달상권,2120240,개롱역,CS300016,안경,64011302,572,제조업
2178586,2022,D,발달상권,2120240,개롱역,CS300001,슈퍼마켓,4169784327,195988,식료품_도소매업


In [310]:
for i in range(est_sale['구분'].nunique()):
    print(est_sale['구분'].unique()[i],len(est_sale[est_sale['구분']==est_sale['구분'].unique()[i]]))

음식점_및_주점업 426176
제조업 277209
기타_도소매업 195831
생활용품_도소매업 104457
수리업 117390
식료품_도소매업 256680
가전제품_및_통신기기_도소매업 65548
스포츠_및_오락_서비스업 218257
전문_서비스업 35781
보건업 131588
교육_및_사업자원_서비스업 117292
개인_서비스업 179291
기타_서비스업 8515
숙박업 39013
부동산_및_임대업 5321
중고상품_도소매업 239


In [311]:
est_sale=est_sale.drop(['서비스_업종_코드','서비스_업종_코드_명'],axis=1)


,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,분기당_매출_금액,분기당_매출_건수,구분
0,2017,A,골목상권,2110001,이북5도청사,816890247,28082,음식점_및_주점업
1,2017,A,골목상권,2110706,그라운드 고척,102416775,116,제조업
2,2017,A,골목상권,2110706,그라운드 고척,12257816,667,기타_도소매업
3,2017,A,골목상권,2110706,그라운드 고척,39851942,1615,기타_도소매업
4,2017,A,골목상권,2110706,그라운드 고척,66164481,1067,기타_도소매업
...,...,...,...,...,...,...,...,...
2178583,2022,D,발달상권,2120240,개롱역,64884644,4268,식료품_도소매업
2178584,2022,D,발달상권,2120240,개롱역,11273198,150,생활용품_도소매업
2178585,2022,D,발달상권,2120240,개롱역,64011302,572,제조업
2178586,2022,D,발달상권,2120240,개롱역,4169784327,195988,식료품_도소매업


In [312]:
est_sale_temp = est_sale.groupby(['기준_년_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명','구분'],as_index=False)[['분기당_매출_금액','분기당_매출_건수']].mean()

In [316]:
est_sale_temp[est_sale_temp['구분']=='음식점_및_주점업'].sort_values(by='분기당_매출_건수',ascending=False).head(10)

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,구분,분기당_매출_금액,분기당_매출_건수
38337,2018,D,발달상권,2120189,강남역,음식점_및_주점업,1.903904e+10,1.103906e+06
17325,2017,D,발달상권,2120189,강남역,음식점_및_주점업,1.773877e+10,1.072849e+06
20961,2017,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,음식점_및_주점업,1.616721e+10,9.376494e+05
41967,2018,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,음식점_및_주점업,1.614801e+10,9.242268e+05
57691,2019,D,발달상권,2120189,강남역,음식점_및_주점업,1.530707e+10,8.900178e+05
60991,2019,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,음식점_및_주점업,1.534318e+10,7.980594e+05
76449,2020,D,발달상권,2120189,강남역,음식점_및_주점업,1.148367e+10,7.027903e+05
116761,2022,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,음식점_및_주점업,1.467409e+10,6.955489e+05
16226,2017,D,발달상권,2120103,홍대입구역(홍대),음식점_및_주점업,1.258850e+10,6.840563e+05
37241,2018,D,발달상권,2120103,홍대입구역(홍대),음식점_및_주점업,1.237396e+10,6.632785e+05


In [318]:
est_sale_temp['분기당_매출_건수'].round()

0            398.0
1            166.0
2             96.0
3           6647.0
4            358.0
            ...   
116808      3301.0
116809    221598.0
116810    220662.0
116811       370.0
116812     86072.0
Name: 분기당_매출_건수, Length: 116813, dtype: float64

In [325]:
# 분기당_매출_건수 소수점 없애기
est_sale_temp['분기당_매출_건수'] =est_sale_temp['분기당_매출_건수'].round()
est_sale_temp['분기당_매출_건수']=est_sale_temp['분기당_매출_건수'].astype(int)

In [327]:
#특수문자 제거
import re
def cleanText(Data): # 괄호와 _을 제외한 모든 특수문자
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\\\[\]\<\>`\'…》]','', Data)
    return text 
temp=[]
for i in est_sale_temp['상권_코드_명']:
    temp.append(cleanText(i))
est_sale_temp['상권_코드_명']=temp
est_sale=est_sale_temp


In [328]:
# 기능상실 적힌 상권코드명 row 제거
drop_list = []
for i in est_sale['상권_코드_명']:
    if('기능상실' in i and i not in drop_list):
        drop_list.append(i)
    else: continue

for i in drop_list:
    est_sale.drop(est_sale[(est_sale['상권_코드_명']==i)].index,inplace = True)

In [ ]:
# 값이 없는 부분을 key error 나타나기 전에 제거
temp=[]
for i in range(len(est_sale)):
    try: 
        print(est_sale['상권_코드_명'][i] ,est_sale['상권_코드'][i]) 
    except KeyError:
        temp.append(i)
for i in temp:
    est_sale.drop(index=est_sale.iloc[i].name)
# index reset
est_sale=est_sale.reset_index(drop=True)

In [330]:
# 빼야 하는 것들 : 전문_서비스업 , 보건업 이렇게 두개
est_sale.drop(est_sale[est_sale['구분']=='전문_서비스업'].index,inplace=True)
est_sale.drop(est_sale[est_sale['구분']=='보건업'].index,inplace=True)

est_sale.reset_index(drop=True,inplace=True)

In [336]:
#est_sale.rename(columns=)
#est_sale[(est_sale['상권_코드_명']=='이북5도청사') & (est_sale['구분']=='제조업')] # and가 아니고 &를 써야함
est_sale.rename(columns={'분기당_매출_금액':'매출_금액','분기당_매출_건수':'매출_건수'},inplace=True)


In [337]:
est_sale

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,구분,매출_금액,매출_건수
0,2017,A,골목상권,2110001,이북5도청사,개인_서비스업,3.230693e+07,398
1,2017,A,골목상권,2110001,이북5도청사,교육_및_사업자원_서비스업,2.660860e+07,166
2,2017,A,골목상권,2110001,이북5도청사,기타_도소매업,2.564806e+06,96
3,2017,A,골목상권,2110001,이북5도청사,생활용품_도소매업,3.070350e+07,358
4,2017,A,골목상권,2110001,이북5도청사,수리업,1.999051e+08,874
...,...,...,...,...,...,...,...,...
101019,2022,U,관광특구,1001496,강남 마이스 관광특구,수리업,4.633577e+09,166562
101020,2022,U,관광특구,1001496,강남 마이스 관광특구,스포츠_및_오락_서비스업,9.380306e+08,3301
101021,2022,U,관광특구,1001496,강남 마이스 관광특구,식료품_도소매업,3.745339e+09,221598
101022,2022,U,관광특구,1001496,강남 마이스 관광특구,음식점_및_주점업,4.562382e+09,220662


#### 추정매출과 점포 dataframe merge  

In [347]:
jumpo_est_sale_inner = pd.merge(jumpo, est_sale, how='inner')
jumpo_est_sale_outer = pd.merge(jumpo, est_sale, how='outer')

In [348]:
jumpo_est_sale_inner

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,구분,점포_수,매출_금액,매출_건수
0,2017,A,골목상권,2110001,이북5도청사,개인_서비스업,116,3.230693e+07,398
1,2017,A,골목상권,2110001,이북5도청사,교육_및_사업자원_서비스업,128,2.660860e+07,166
2,2017,A,골목상권,2110001,이북5도청사,기타_도소매업,98,2.564806e+06,96
3,2017,A,골목상권,2110001,이북5도청사,생활용품_도소매업,151,3.070350e+07,358
4,2017,A,골목상권,2110001,이북5도청사,수리업,24,1.999051e+08,874
...,...,...,...,...,...,...,...,...,...
99866,2022,U,관광특구,1001496,강남 마이스 관광특구,수리업,319,4.633577e+09,166562
99867,2022,U,관광특구,1001496,강남 마이스 관광특구,스포츠_및_오락_서비스업,9,9.380306e+08,3301
99868,2022,U,관광특구,1001496,강남 마이스 관광특구,식료품_도소매업,140,3.745339e+09,221598
99869,2022,U,관광특구,1001496,강남 마이스 관광특구,음식점_및_주점업,549,4.562382e+09,220662


In [349]:
jumpo_est_sale_outer

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,구분,점포_수,매출_금액,매출_건수
0,2017,A,골목상권,2110001,이북5도청사,개인_서비스업,116.0,3.230693e+07,398.0
1,2017,A,골목상권,2110001,이북5도청사,교육_및_사업자원_서비스업,128.0,2.660860e+07,166.0
2,2017,A,골목상권,2110001,이북5도청사,기타_도소매업,98.0,2.564806e+06,96.0
3,2017,A,골목상권,2110001,이북5도청사,기타_서비스업,4.0,NaN,NaN
4,2017,A,골목상권,2110001,이북5도청사,부동산_및_임대업,156.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
127361,2022,R,전통시장,2130136,수유재래시장,스포츠_및_오락_서비스업,NaN,1.500000e+06,20.0
127362,2022,R,전통시장,2130176,유진상가,스포츠_및_오락_서비스업,NaN,7.040224e+07,5114.0
127363,2022,R,전통시장,2130190,홍대소상공인상점가,숙박업,NaN,7.089559e+07,136.0
127364,2022,R,전통시장,2130258,영진시장A동,생활용품_도소매업,NaN,1.617516e+06,32.0


##### 나머지 4개 inner, outer join 결과  

In [352]:
data = pd.merge(living_pop, jipgaek, how='inner')
data = pd.merge(data, apt, how='inner')
data = pd.merge(data, change_factor, how='inner')
data


,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,집객시설_수,아파트_평균_면적,아파트_평균_시가,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,상권확장,83.75,60.25,98.75,51.25
1,2017,A,골목상권,2110002,독립문역 1번,314832.50,31.5,78.0,311503096.0,HH,정체,117.75,55.00,98.75,51.25
2,2017,A,골목상권,2110003,세검정초등학교,361826.00,48.0,89.0,223873171.0,HH,정체,107.75,55.00,98.75,51.25
3,2017,A,골목상권,2110004,대신고등학교,331062.00,44.5,50.5,122651126.0,HH,정체,153.25,68.25,98.75,51.25
4,2017,A,골목상권,2110005,세검정,218436.75,44.5,88.5,225876911.5,HH,정체,105.75,56.50,98.75,51.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10214,2022,R,전통시장,2130325,명일전통시장,208207.25,2.0,61.0,294384895.0,HL,상권축소,129.25,52.75,121.25,55.50
10215,2022,U,관광특구,1001491,이태원 관광특구,2008554.00,63.0,79.0,439233880.0,HH,정체,129.75,61.00,121.25,55.50
10216,2022,U,관광특구,1001493,동대문패션타운 관광특구,3129478.00,149.0,45.0,259406778.0,HH,정체,147.00,64.00,121.25,55.50
10217,2022,U,관광특구,1001494,종로청계 관광특구,3864423.00,243.0,37.0,141690132.0,HH,정체,162.50,74.75,121.25,55.50


In [353]:
data = pd.merge(data, jumpo_est_sale_inner, how='inner')
data


,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,집객시설_수,아파트_평균_면적,아파트_평균_시가,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,구분,점포_수,매출_금액,매출_건수
0,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,상권확장,83.75,60.25,98.75,51.25,개인_서비스업,116,3.230693e+07,398
1,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,상권확장,83.75,60.25,98.75,51.25,교육_및_사업자원_서비스업,128,2.660860e+07,166
2,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,상권확장,83.75,60.25,98.75,51.25,기타_도소매업,98,2.564806e+06,96
3,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,상권확장,83.75,60.25,98.75,51.25,생활용품_도소매업,151,3.070350e+07,358
4,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,상권확장,83.75,60.25,98.75,51.25,수리업,24,1.999051e+08,874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106936,2022,U,관광특구,1001495,잠실 관광특구,3469611.00,149.0,74.0,552320698.0,LL,다이나믹,117.00,54.25,121.25,55.50,숙박업,288,1.781418e+09,44120
106937,2022,U,관광특구,1001495,잠실 관광특구,3469611.00,149.0,74.0,552320698.0,LL,다이나믹,117.00,54.25,121.25,55.50,스포츠_및_오락_서비스업,373,4.463282e+08,9601
106938,2022,U,관광특구,1001495,잠실 관광특구,3469611.00,149.0,74.0,552320698.0,LL,다이나믹,117.00,54.25,121.25,55.50,식료품_도소매업,503,5.638079e+09,805540
106939,2022,U,관광특구,1001495,잠실 관광특구,3469611.00,149.0,74.0,552320698.0,LL,다이나믹,117.00,54.25,121.25,55.50,음식점_및_주점업,2210,6.571774e+09,265556


In [358]:
data2 = pd.merge(living_pop, jipgaek, how='outer')
data2= pd.merge(data2, apt, how='outer')
data2 = pd.merge(data2, change_factor, how='outer')
data2

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,집객시설_수,아파트_평균_면적,아파트_평균_시가,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2017,A,골목상권,1000001,계동길,337476.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,A,골목상권,1000002,난계로27길,1026880.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,A,골목상권,1000003,돈화문로11가길,801396.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,A,골목상권,1000004,명륜길,759851.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,A,골목상권,1000005,백석동길,23389.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12841,2022,R,전통시장,2130321,장신구 특화거리 상점가,NaN,7.0,43.0,208066698.0,LL,다이나믹,112.666667,55.00,122.00,55.666667
12842,2022,R,전통시장,2130326,고덕 골목형상점가,NaN,11.0,53.0,633301800.0,LH,상권확장,77.500000,62.50,121.25,55.500000
12843,2019,R,전통시장,2130258,영진시장A동,NaN,NaN,88.0,585308360.0,LH,상권확장,96.500000,63.25,108.50,53.250000
12844,2017,R,전통시장,2130040,남평화시장,NaN,NaN,NaN,NaN,HH,정체,100.750000,90.75,98.75,51.250000


In [359]:
data2= pd.merge(data2, jumpo_est_sale_outer, how='outer')
data2


,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,집객시설_수,아파트_평균_면적,아파트_평균_시가,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,구분,점포_수,매출_금액,매출_건수
0,2017,A,골목상권,1000001,계동길,337476.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,A,골목상권,1000002,난계로27길,1026880.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,A,골목상권,1000003,돈화문로11가길,801396.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,A,골목상권,1000004,명륜길,759851.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,A,골목상권,1000005,백석동길,23389.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146366,2021,U,관광특구,1001494,종로·청계 관광특구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,숙박업,NaN,4.936814e+08,12349.0
146367,2021,U,관광특구,1001494,종로·청계 관광특구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,스포츠_및_오락_서비스업,NaN,2.258016e+08,17174.0
146368,2021,U,관광특구,1001494,종로·청계 관광특구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,식료품_도소매업,NaN,1.823538e+09,167395.0
146369,2021,U,관광특구,1001494,종로·청계 관광특구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,음식점_및_주점업,NaN,4.992905e+09,241766.0


In [361]:
print(data.shape)
print(data2.shape)

(106941, 19)
(146371, 19)


In [362]:
data.to_csv('/Users/junny/Downloads/쭌/쭌_학교관련/23_1_수업/캡디/5주차/KimJuneHyun_EDA/preprocessed_data/commercial_inner.csv',index=None)
data2.to_csv('/Users/junny/Downloads/쭌/쭌_학교관련/23_1_수업/캡디/5주차/KimJuneHyun_EDA/preprocessed_data/commercial_outer.csv',index=None)

In [1]:
# 전처리가 되어야하는 방향성을 찾아보자면
#  일단 분기마다 같은 data 를 mean으로 묶어줘야함..
# 0. 특정 자격이 필요한 직업들 빼기
# 1. 내가 나눈 상권의 분류대로 저놈들을 싹 나누고
# 2. 고려될 요소가 더 있는지 확인
# 3. 그리고 공실률을 대신해서 사용할 요소가 있는지?? -> 이거 녹음 들어보셈
# 4. 변수 좀 줄이기 
# 5. 교수님이 알려준 자료, 공실률 데이터를 지역별로 엮을 수 있는지?? 확인하기
# 6. 공실률을 어떻게 묶을것인가?